# Introduction

"""
@author: ArturKovalchuk
"""

**
IMPORTANT:
works only with Python 3.5**
1. Install Magic Wand (need older version)
available here: https://ftp.icm.edu.pl/packages/ImageMagick/binaries/
    ImageMagick-6.9.8-9-Q16-x64-dll.exe
2. Do this:
    http://docs.wand-py.org/en/latest/guide/install.html#install-imagemagick-on-windows
or here: http://docs.wand-py.org/en/0.2-maintenance/_images/windows-envvar.png
3. From Conda install Wand:
    pip install Wand
4. https://github.com/UB-Mannheim/tesseract/wiki    and here    https://github.com/UB-Mannheim/tesseract/wiki

https://ghostscript.com/Ghostscript_9.22.html

Additional info:

https://gist.github.com/dannguyen/a0b69c84ebc00c54c94d

https://testpypi.python.org/pypi/pytesseract

https://github.com/tesseract-ocr/tesseract/wiki

https://github.com/UB-Mannheim/tesseract/wiki


In order to create new environment, You need to run these commands in the console:
- **conda create -n python_3.5 python=3.5 anaconda** # creates new environment with Python 3.5
- **pip install wand** # installs wand for converting pdf to jpg
- **pip install pytesseract** # installs pytesseract for OCR

**Activate Google Vision API, including credentials**

# Body

### Check the environment we are working in:

In [91]:
#Check the environment python version
import sys
print(sys.version)

3.6.3 |Anaconda, Inc.| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]


### Import the liabraries we need:

In [92]:
# for working with files    
import io
import os

# for data manipulation
import pandas as pd

# for downloading
import requests

# for converting pdf to jpg with processing
from wand.image import Image as wImage

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types    

# IMPORTANT: activate the credentials for Google API
# Instantiates a client
client = vision.ImageAnnotatorClient()#creds=json.load(open('Prozorro_prices-febfc26c2945.json')) #credentials=creds


# import regular expressions
import re

### Create the functions we need:

In [93]:
def download_contract(url, contract_id):
    """The unction takes contract's url and id, and downloads conttract to the disc"""
    response = requests.get(url)

    file_name =contract_id + ".pdf"
    with open(file_name, 'wb') as f:
         f.write(response.content)
    return file_name

In [94]:
def convert_pdf_to_single_page_img(file_name, start_page=0, end_page=0):  
    """Function takes pdf file and converts it to the single-page image.
        Input:
            file_name - name of the pdf-file;
            start_page - starting page of the pdf-file to convert;
            end_page - final page of the pdf-file to convert.
        Output:
            name of the single-page image file saved.
    """
    with wImage(filename=file_name, resolution=300) as img:
        width_pdf=max([image.width for image in img.sequence]) # width of the document
        height_pdf=sum([image.height for image in img.sequence]) # lenght of the document
        
        with wImage() as blankimage:
            blankimage.blank(width=width_pdf, height=height_pdf)
            
            width_blank=0; height_blank=0
            for page_num in range(start_page, len(img.sequence)-end_page):
                img_to_save=wImage(img.sequence[page_num])
                img_to_save.threshold(0.7)
                img_to_save.compression_quality = 100
                img_to_save.format='png'
                
                blankimage.composite(img_to_save, width_blank, height_blank)
                height_blank+=img_to_save.height
            
            blankimage.save(filename = file_name+'.png')
        
    return file_name+'.png'

In [95]:
# function to iterate through text lines
def lines_iteration(text): 
    return iter(text.splitlines())

### Main code

#### Import the data on contracts

Create the list of contracts to read from the **txt** file:

In [96]:
contracts=open("test_gas.txt").read()
contracts=list(lines_iteration(contracts))

contracts_list=[]
for contract in contracts:
    contracts_list.append(contract.split("    "))

contracts=pd.DataFrame(contracts_list)
contracts.columns=["contract_id", "contract_url", "contract_name", "contract_format"]

contracts=contracts[contracts["contract_format"]=="application/pdf"]

contract_id_list=list(contracts["contract_id"])
contract_url_list=list(contracts["contract_url"])

contracts_id_url=dict(zip(contract_id_list, contract_url_list))

Create the list of contracts to read from the **excel** file:

In [97]:
# contracts=pd.read_excel("test_gas.xlsx")
# contracts.columns=["contract_id", "contract_url"]

# contract_id_list=list(contracts["contract_id"])
# contract_url_list=list(contracts["contract_url"])

# contracts_id_url=dict(zip(contract_id_list, contract_url_list))

#### Download and convert the text:

In [98]:
%%time

documents_all={}

for contract_id, contract_url in contracts_id_url.items(): # iterate over contracts
    contract_pdf=download_contract(contract_url, contract_id) # download the files, returns the file's name
    contract_image=convert_pdf_to_single_page_img(contract_pdf, start_page=0, end_page=0) # converts downloaded file into images
    
    
    # Loads the image into memory
    with io.open(contract_image, 'rb') as image_file:
        content = image_file.read()
        image = types.Image(content=content)

    response = client.document_text_detection(image=image)
    document = response.full_text_annotation
    try:
        contract_text=response.text_annotations[0].description
        documents_all[contract_id]=contract_text
    except IndexError:
        pass    

Wall time: 4min 23s


**Formatted text:**

In [99]:
for contract in list(documents_all.values()):
    print('\n"{}"'.format(contract))
    print("==========================================================================================================")


"ДОГОВІР N%41DPDрz2429-17
на постачання природного газу
м. Павлоград
«01» листопада 2017 р.
ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ДНІПРОПЕТРОВСЬКГАЗ ЗБУТ»,
ЕІС-код 56X930000000060В, що здійснює діяльність на підставі ліцензії на право провадження господарської
діяльності з постачання природного газу на території України, затвердженої Постановою НКРЕКП від
16.05.2011 року №653, далі — Постачальник, в особі начальника Павлоградського відділу реалізації
Колесника Сергія Сергійовича, який діє на підставі довіреності від 20.12.2016 року, з однієї сторони, і
Дніпровське приміське об'єднане управління Пенсійного фонду України Дніпропетровської
області, EIC код 56XS000156TRЈ008, далі . Споживач, в особі заступника начальника Купріна
Олександра Васильовича, який діє на підставі Положення, з другої сторони, а разом поіменовані сторони,
уклали цей договір на постачання природного газу (далі - Договір) на наведених нижче умовах.
Найменування Оператора газорозподільної системи, далі — Оператор 

#### Constructing DataFrame with prices:

In [100]:
df_prices=pd.DataFrame(columns=["price_without_VAT", "VAT", "price_with_VAT"])
price_with_VAT={}

for key, line_with_price in text_with_price.items():
    merged_line=line_with_price[4:].replace(" ", "").replace("1000куб.м", "").replace(",", ".")
    #numbers_extracted=re.findall('\d+\.\d+', merged_line)
    numbers_extracted=[float(number) for number in re.findall('\d+\.\d+', merged_line)]
    print(line_with_price)
    print(merged_line)
    print(numbers_extracted)
    print()
    if ("зпдв" in merged_line.lower()) | ("зпдb" in merged_line.lower()): price_with_VAT[key]=max(numbers_extracted)

df_prices["price_with_VAT"]=pd.Series(price_with_VAT)
df_prices

3.2. Ціна газу становить 8 479,10 грн. за 1000 куб. м, крім того ПДВ 1 695,82 грн., всього з ПдB – 10 174,92
Цінагазустановить8479.10грн.за.крімтогоПДВ1695.82грн..всьогозПдB–10174.92
[8479.1, 1695.82, 10174.92]

3.2. Ціна газу становить 9 483,00 грн. (дев'ять тисяч чотириста вісімдесят три грн. 00 коп.) з ПдB за 1000 куб. м.
Цінагазустановить9483.00грн.(дев'ятьтисяччотириставісімдесяттригрн.00коп.)зПдBза.
[9483.0]



,price_without_VAT,VAT,price_with_VAT
66932d4cc5944f3a9c5dfcb4572f804d,NaN,NaN,10174.92
67bc36369b124e968fe1fff68b401155,NaN,NaN,9483.00


#### Preliminary results

In [101]:
text_with_price={}

for key, contract in documents_all.items():
    contract_raw_list=list(lines_iteration(contract))
    for line in contract_raw_list:
        if "ціна газу становить" in line.lower():
            text_with_price[key]=line

**Here are the the rows that consist price mentioned:**

{{for element in text_with_price.values(): print(element+"\n")}}

**Preliminary efficiency resuls:**

- **{{len(documents_all)}} out of {{len(contracts)}}** documents (or **{{100*round(len(documents_all)/len(contracts),1)}}%**) were read.
- **{{len(text_with_price)}} out of {{len(documents_all)}}** documents (or **{{100*round(len(text_with_price)/len(documents_all),1)}}%**) consist prices.

# Reading

- https://cloud.google.com/vision/docs/fulltext-annotations

Text representation:
- https://cloud.google.com/vision/docs/detecting-text


In [8]:
# Loads the image into memory
with io.open(contract_image, 'rb') as image_file:
    content = image_file.read()
    image = types.Image(content=content)


In [9]:
%%time
response = client.document_text_detection(image=image)
document = response.full_text_annotation#response.text_annotations[0]

Wall time: 16.6 s


In [10]:
contract_text=response.text_annotations[0].description#["description"]
contract_text

',\t,\nдогоBIP { ABн мz S - 41\nна постачання природного газу для потреб непобутових споживачів\nм, Хмельницький\n34 30, До{1 р.\nтоВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ХМЕЛЬНИЦЬКГАЗ ЗБУТ», ЕIC код 56X930000000280Ү, що здійснює\nдіяльність на підставі ліцензії на право провадження господарської діяльності з постачання природного газу на території України, затвердженої\nпостановою НКРЕКП від 1.05.2017р. № 633, далі — Постачальник, в особі начальника відділу супроводу клієнтів Буняк Оксани Юріївни, яка діє на\nпідставі довіреності і Др-10-0317 від 06.03.2017р, з однієї сторони, і цpouis CLлітохо .\nочеЫщаба маталаргуархітекчи , шіюбанья и\nрутекчи ісsas on 1 льұ\nKakciЫ\nEI€ код Зе хgoo00 був 600. , далі - Споживач, в особі Олоццо Тек ада:4\nякий діє на підставі очія о ударя\n, з другої сторони, а разом поіменовані сторони, уклали цей\nдоговір на постачання природного газу (далі - договір) на наведених нижче умовах.\nНайменування Оператора газорозподільної системи, далі — Оператор ГРМ,

Pretty text:

In [11]:
print('\n"{}"'.format(contract_text))


",	,
догоBIP { ABн мz S - 41
на постачання природного газу для потреб непобутових споживачів
м, Хмельницький
34 30, До{1 р.
тоВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ХМЕЛЬНИЦЬКГАЗ ЗБУТ», ЕIC код 56X930000000280Ү, що здійснює
діяльність на підставі ліцензії на право провадження господарської діяльності з постачання природного газу на території України, затвердженої
постановою НКРЕКП від 1.05.2017р. № 633, далі — Постачальник, в особі начальника відділу супроводу клієнтів Буняк Оксани Юріївни, яка діє на
підставі довіреності і Др-10-0317 від 06.03.2017р, з однієї сторони, і цpouis CLлітохо .
очеЫщаба маталаргуархітекчи , шіюбанья и
рутекчи ісsas on 1 льұ
KakciЫ
EI€ код Зе хgoo00 був 600. , далі - Споживач, в особі Олоццо Тек ада:4
який діє на підставі очія о ударя
, з другої сторони, а разом поіменовані сторони, уклали цей
договір на постачання природного газу (далі - договір) на наведених нижче умовах.
Найменування Оператора газорозподільної системи, далі — Оператор ГРМ, з яким Споживач

## Test

In [ ]:
# contract_text_list=contract_text.split('\n')
# contract_text_list

Texts:

",	,
догоBIP { ABн мz S - 41
на постачання природного газу для потреб непобутових споживачів
м, Хмельницький
34 30, До{1 р.
тоВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ «ХМЕЛЬНИЦЬКГАЗ ЗБУТ», ЕIC код 56X930000000280Ү, що здійснює
діяльність на підставі ліцензії на право провадження господарської діяльності з постачання природного газу на території України, затвердженої
постановою НКРЕКП від 1.05.2017р. № 633, далі — Постачальник, в особі начальника відділу супроводу клієнтів Буняк Оксани Юріївни, яка діє на
підставі довіреності і Др-10-0317 від 06.03.2017р, з однієї сторони, і цpouis CLлітохо .
очеЫщаба маталаргуархітекчи , шіюбанья и
рутекчи ісsas on 1 льұ
KakciЫ
EI€ код Зе хgoo00 був 600. , далі - Споживач, в особі Олоццо Тек ада:4
який діє на підставі очія о ударя
, з другої сторони, а разом поіменовані сторони, уклали цей
договір на постачання природного газу (далі - договір) на наведених нижче умовах.
Найменування Оператора газорозподільної системи, далі — Оператор ГРМ, з яким С

"правових"

"актів"

"."

"V"

"."

"Права"

"та"

"обов"

"'"

"язки"

"Сторін"

"5"

"."

"1"

"."

"Постачальник"

"має"

"право"

":"

"5"

"."

"1"

"."

"1"

"."

"Отримувати"

"від"

"Споживача"

"оплату"

"поставленого"

"газу"

"відповідно"

"до"

"умов"

"розділів"

"("

"II"

","

"IV"

"Договору"

"."

"5"

"."

"1"

"."

"2"

"."

"Безперешкодного"

"доступу"

"на"

"територію"

"об"

"'"

"єктів"

"Споживача"

","

"а"

"також"

"до"

"комерційних"

"вузлів"

"обліку"

"природного"

"газу"

","

"що"

"встановлені"

"на"

"об"

"'"

"єктах"

"Споживача"

","

"для"

"звірки"

"даних"

"фактичного"

"споживання"

"природного"

"газу"

"."

"Доступ"

"здійснюється"

"працівниками"

"("

"представниками"

")"

"Постачальника"

"за"

"пред"

"'"

"явленням"

"службового"

"посвідчення"

"("

"довіреності"

")"

"."

"5"

"."

"1"

"."

"3"

"."

"Отримувати"

"повну"

"і"

"достовірну"

"інформацію"

"від"

"Споживача"

"щодо"

"режимів"

"споживання"

"природного"

"газу"

"


"недовідпущеного"

"газу"

","

"який"

"обчислюється"

","

"виходячи"

"з"

"підтвердженого"

"обсягу"

"газу"

"на"

"відповідний"

"період"

"з"

"урахуванням"

"періоду"

"безпідставного"

"припинення"

"газопостачання"

","

"та"

"вартість"

"робіт"

"з"

"припинення"

"і"

"повторного"

"відновлення"

"подачі"

"газу"

"після"

"його"

"безпідставного"

"припинення"

"."

"6"

"."

"3"

"."

"3"

"."

"У"

"разі"

","

"якщо"

"підтверджений"

"обсяг"

"газу"

"буде"

"менше"

"планового"

"обсягу"

","

"визначеного"

"договором"

","

"або"

"несвоєчасно"

"погоджений"

"Постачальником"

"з"

"Оператором"

"ГTC"

"на"

"відповідний"

"період"

"("

"за"

"умови"

","

"що"

"Споживачем"

"не"

"порушувались"

"зобов"

"'"

"язання"

"за"

"Договором"

")"

","

"що"

"призвело"

"до"

"припинення"

"розподілу"

"природного"

"газу"

"Оператором"

"ГРМ"

","

"Споживач"

"має"

"право"

"вимагати"

"від"

"постачальника"

"відшкодування"

"вартості"

"або"

"об"

"'"

"єму"



"скріплюються"

"їх"

"печатками"

"("

"за"

"наявності"

")"

"."

"11"

"."

"5"

"."

"Сторони"

"зобов"

"'"

"язуються"

"письмово"

"повідомляти"

"про"

"зміну"

"реквізитів"

"("

"місцезнаходження"

","

"найменування"

","

"організаційно"

"-"

"правової"

"форми"

","

"банківських"

"реквізитів"

"тощо"

")"

"не"

"пізніше"

"ніж"

"через"

"десять"

"календарних"

"днів"

"після"

"настання"

"таких"

"змін"

"."

"("

"1"

"."

"6"

"."

"Постачальник"

"має"

"статус"

"платника"

"податку"

"на"

"прибуток"

"на"

"загальних"

"умовах"

","

"передбачених"

"Податковим"

"Кодексом"

"України"

"і"

"статус"

"платника"

"податку"

"на"

"додану"

"вартість"

"на"

"загальних"

"умовах"

","

"передбачених"

"Податковим"

"Кодексом"

"України"

"."

"11"

"."

"7"

"."

"Характеристика"

"статусу"

"Споживача"

","

"як"

"платника"

"податків"

":"

"11"

"."

"7"

"."

"1"

"."

"Споживач"

"€"

"("

"не"

"є"

")"

"платником"

"|"

"1"

"."

"7"

"."

"2"

"."

"С

In [ ]:
contract_text_list_part=[]
for index in len(contract_text_list):
    if "Ціна постачання природного газу" in contract_text_list[index]:
        

In [14]:
for page in document.pages:
    for block in page.blocks:
        block_words = []
        for paragraph in block.paragraphs:
            block_words.extend(paragraph.words)

        block_symbols = []
        for word in block_words:
            block_symbols.extend(word.symbols)

        block_text = ''
        for symbol in block_symbols:
            block_text = block_text + symbol.text

        print('Block Content: {}'.format(block_text))
        #print('Block Bounds:\n {}'.format(block.bounding_box))

Block Content: ДОГОВІРN%41DPDрz2429-17напостачанняприродногогазу
Block Content: м.Павлоград
Block Content: «01»листопада2017р.
Block Content: ТОВАРИСТВОЗОБМЕЖЕНОЮВІДПОВІДАЛЬНІСТЮ«ДНІПРОПЕТРОВСЬКГАЗЗБУТ»,ЕІС-код56X930000000060В,щоздійснюєдіяльністьнапідставіліцензіїнаправопровадженнягосподарськоїдіяльностізпостачанняприродногогазунатериторіїУкраїни,затвердженоїПостановоюНКРЕКПвід16.05.2011року№653,далі—Постачальник,вособіначальникаПавлоградськоговідділуреалізаціїКолесникаСергіяСергійовича,якийдієнапідставідовіреностівід20.12.2016року,зоднієїсторони,і
Block Content: Дніпровськеприміськеоб'єднанеуправлінняПенсійногофондуУкраїниДніпропетровськоїобласті,EICкод56XS000156TRЈ008,далі.Споживач,вособізаступниканачальникаКупрінаОлександраВасильовича,якийдієнапідставіПоложення,здругоїсторони,аразомпоіменованісторони,уклалицейдоговірнапостачанняприродногогазу(далі-Договір)нанаведенихнижчеумовах.
Block Content: НайменуванняОператорагазорозподільноїсистеми,далі—ОператорГРМ,зякимСпоживачуклавдоговірро

In [78]:
# for page in document.pages:
#     for block in page.blocks:
#         block_words = []
#         for paragraph in block.paragraphs:
#             block_words.extend(paragraph.words)

#         block_symbols = []
#         for word in block_words:
#             block_symbols.extend(word.symbols)

#         block_text = ''
#         for symbol in block_symbols:
#             block_text = block_text + symbol.text

#         print('Block Content: {}'.format(block_text))
#         #print('Block Bounds:\n {}'.format(block.bounding_box))

In [80]:
paragraph.words

[property {
  detected_languages {
    language_code: "en"
  }
}
bounding_box {
  vertices {
    x: 634
    y: 13738
  }
  vertices {
    x: 636
    y: 13736
  }
  vertices {
    x: 645
    y: 13747
  }
  vertices {
    x: 642
    y: 13749
  }
}
symbols {
  property {
    detected_languages {
      language_code: "en"
    }
  }
  bounding_box {
    vertices {
      x: 634
      y: 13736
    }
    vertices {
      x: 645
      y: 13736
    }
    vertices {
      x: 645
      y: 13749
    }
    vertices {
      x: 634
      y: 13749
    }
  }
  text: ":"
}
, property {
  detected_languages {
    language_code: "en"
  }
}
bounding_box {
  vertices {
    x: 655
    y: 13723
  }
  vertices {
    x: 657
    y: 13721
  }
  vertices {
    x: 666
    y: 13732
  }
  vertices {
    x: 663
    y: 13734
  }
}
symbols {
  property {
    detected_languages {
      language_code: "en"
    }
  }
  bounding_box {
    vertices {
      x: 655
      y: 13721
    }
    vertices {
      x: 666
      y: 13721

In [79]:
for page in document.pages:
    for block in page.blocks:
        block_words = []
        for paragraph in block.paragraphs:
            block_words.extend(paragraph.words)

        block_symbols = []
        for word in block_words:
            block_symbols.extend(word.symbols)

        block_text = ''
        for symbol in block_symbols:
            block_text = block_text + symbol.text

        print('Block Content: {}'.format(block_text))
        print('Block Bounds:\n {}'.format(block.bounding_box))

Block Content: ,,
Block Bounds:
 vertices {
  x: 594
  y: 14
}
vertices {
  x: 697
  y: 14
}
vertices {
  x: 697
  y: 65
}
vertices {
  x: 594
  y: 65
}

Block Content: догоBIP{ABнмzS-41напостачанняприродногогазудляпотребнепобутовихспоживачів
Block Bounds:
 vertices {
  x: 750
  y: 56
}
vertices {
  x: 1844
  y: 52
}
vertices {
  x: 1844
  y: 157
}
vertices {
  x: 750
  y: 161
}

Block Content: м,Хмельницький
Block Bounds:
 vertices {
  x: 177
  y: 155
}
vertices {
  x: 415
  y: 154
}
vertices {
  x: 415
  y: 178
}
vertices {
  x: 177
  y: 179
}

Block Content: 3430,До{1р.
Block Bounds:
 vertices {
  x: 2017
  y: 135
}
vertices {
  x: 2359
  y: 134
}
vertices {
  x: 2359
  y: 196
}
vertices {
  x: 2017
  y: 197
}

Block Content: тоВАРИСТВОЗОБМЕЖЕНОЮВІДПОВІДАЛЬНІСТЮ«ХМЕЛЬНИЦЬКГАЗЗБУТ»,ЕICкод56X930000000280Ү,щоздійснюєдіяльністьнапідставіліцензіїнаправопровадженнягосподарськоїдіяльностізпостачанняприродногогазунатериторіїУкраїни,затвердженоїпостановоюНКРЕКПвід1.05.2017р.№633,далі—Постача

In [76]:
block_words

In [77]:
block_symbols

[]

In [57]:
block_text

'::,::..'

# Junk

In [ ]:
table_raw=[]

list_of_goods=["a-95", "дизельне паливо", "a-92"] # list of goods we are looking for

for contract_id, contract_url in contracts_id_url.items(): # iterate over contracts
    file_name=download_contract(contract_url, contract_id) # download the files, returns the file's name
    picture_names=convert_pdf_to_img(file_name, last_page=True) # converts downloaded file into images
    for picture in picture_names: # iterate over images of the contract
        text_raw=pytesseract.image_to_string(Image.open(picture), lang="ukr").lower() # convert image into the text
        try:
            text_raw=text_raw.split("специфікація")[1]
        except IndexError: # break if there is no specification
            break
        text_raw_list=list(lines_iteration(text_raw)) # converts text into the list of rows
        print("Picture's name: ", picture)
        print(text_raw_list)
        print("________________")
        print("________________")
        text_w_goods_list=[]
        for good in list_of_goods:
            for line in text_raw_list:
                if good in line:
                    text_w_goods_list.append([line])
    table_raw.append(text_w_goods_list)
table_raw